# Assignment 12

### 1.复习上课内容

### 2.回答以下理论问题

#### 1. 请写一下TF-IDF的计算公式

tf表示单词在一个句子或文档中出现的次数，idf=log_10⁡(𝑁/(𝑑𝑓_𝑡 ))，如果一个词再很多句子或文档中出现则不重要。
TF-IDF=tf×idf

#### 2. LDA算法的基本假设是什么？

假设1：每个文档的主题服从一个概率分布；
假设2：每个主题的词也服从一个分布。

#### 3. 在TextRank算法中构建图的权重是如何得到的？

两个词向量之间余弦相似度即构建图的权重。

#### 4. 什么是命名实体识别？ 有什么应用场景？

从一句话中识别出人名，地名，组织名，日期时间，这就是命名实体识别的一个例子，而人名，地名等这些被识别的目标就是命名实体。命名实体识别是多分类任务，应用场景：从一句话中识别出人名、地名，从电商的搜索中识别出产品的名字，识别药物名称等等。

#### 5.NLP主要有哪几类任务 ？

分类任务：包括文本分类、情感分析等；

生成任务：包括机器翻译、问答系统（常识问答和阅读理解、图像问答等）；

文本摘要：包括抽取式和生成式，抽取式可能会造成语意不连贯，生成式则可能不稳定。

### 3.实践题

#### 3.1 手动实现TextRank算法 (在新闻数据中随机提取100条新闻训练词向量和做做法测试）

 提示：
 1. 确定窗口，建立图链接。   
 2. 通过词向量相似度确定图上边的权重
 3. 根据公式实现算法迭代(d=0.85)

In [18]:
import gensim
import pandas as pd
import re
from gensim import corpora, models
import jieba.posseg as pseg

In [12]:
def token(string):
    return re.findall(r'[\d|\w]+',string)

content = pd.read_csv('xinhua_news.csv',encoding='gb18030')
articles = content['content'][:100].tolist()

In [43]:
stop_words = []
with open('baidu_stopwords.txt',encoding='utf-8') as f:
    for word in f.readlines():
        stop_words.append(word.strip())

In [52]:
flags = ['n', 'nr', 'ns', 'nt', 'eng', 'v', 'd']
with open('dict_news','w',encoding='gb18030') as n:
    for i in range(len(articles)):
        article =' '.join(token(str(articles[i])))
        article = article.strip() 
        seg_words = pseg.cut(article)
            #print(article)
        article = ' '.join(s.word for s in seg_words if s.flag in flags and s.word not in stop_words) 
        n.write(article+'\n')

In [82]:
from gensim.models import word2vec,Word2Vec
#sentence = word2vec.LineSentence('dict_news.txt')
sentences = word2vec.Text8Corpus('dict_news1.txt')
model=word2vec.Word2Vec(sentences,size=300,window=5,min_count=0)
model.save('model_gram.model')

In [83]:
model=Word2Vec.load('model_gram.model')

In [93]:
import numpy as np

class TextRank(object):
    
    def __init__(self, sentence, window, alpha, iternum):
        self.sentence = sentence
        self.window = window
        self.alpha = alpha
        self.edge_dict = {} #记录节点的边连接字典
        self.iternum = iternum#迭代次数
 
    #对句子进行分词
    def cutSentence(self):
        jieba.load_userdict('dict_news1.txt')
        tag_filter = ['n', 'nr', 'ns', 'nt', 'eng', 'v', 'd']
        seg_result = pseg.cut(self.sentence)
        self.word_list = [s.word for s in seg_result if s.flag in tag_filter and s.word not in stop_words]
        #print(self.word_list)
 
    #根据窗口，构建每个节点的相邻节点,返回边的集合
    def createNodes(self):
        tmp_list = []
        word_list_len = len(self.word_list)
        for index, word in enumerate(self.word_list):
            if word not in self.edge_dict.keys():
                tmp_list.append(word)
                tmp_set = set()
                left = index - self.window + 1#窗口左边界
                right = index + self.window#窗口右边界
                if left < 0: left = 0
                if right >= word_list_len: right = word_list_len
                for i in range(left, right):
                    if i == index:
                        continue
                    tmp_set.add(self.word_list[i])
                self.edge_dict[word] = tmp_set
 
    #根据边的相连关系，构建矩阵
    def createMatrix(self):
        self.matrix = np.zeros([len(set(self.word_list)), len(set(self.word_list))])
        self.word_index = {}#记录词的index
        self.index_dict = {}#记录节点index对应的词
 
        for i, v in enumerate(set(self.word_list)):
            self.word_index[v] = i
            self.index_dict[i] = v
        for key in self.edge_dict.keys():
            for w in self.edge_dict[key]:
                self.matrix[self.word_index[key]][self.word_index[w]] = model.wv.similarity(key,w)
                self.matrix[self.word_index[w]][self.word_index[key]] = model.wv.similarity(key,w)
        #print('matrix',self.matrix)
        
        
    #根据textrank公式计算权重
    def calPR(self):
        self.PR = np.ones([len(set(self.word_list)), 1])
        for i in range(self.iternum):
            self.PR = (1 - self.alpha) + self.alpha * np.dot(self.matrix, self.PR)
 
    #输出词和相应的权重
    def printResult(self):
        word_pr = {}
        for i in range(len(self.PR)):
            word_pr[self.index_dict[i]] = self.PR[i][0]
        res = sorted(word_pr.items(), key = lambda x : x[1], reverse=True)
        print(res[:10])
 
if __name__ == '__main__':
    #for i in range(len(articles))
    print(articles[3])
    s = articles[3]
    tr = TextRank(s, 3, 0.85, 2000)
    tr.cutSentence()
    tr.createNodes()
    tr.createMatrix()
    tr.calPR()
    tr.printResult()

这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄

[('汽车', 0.2821610740599017), ('记者', 0.26283673671530094), ('地区', 0.19240165513938956), ('烧毁', 0.1739810275917192), ('佩德罗', 0.17151191169026292), ('拍摄', 0.17130581891236818), ('新华社', 0.1695974340778483), ('大火', 0.1648526969559842), ('葡萄牙', 0.1599515697182538), ('张立云', 0.14997005290940393)]


#### 选做 1.  提取新闻人物里的对话。(使用以上提取小数据即可）

提示：    
1.寻找预料里具有表示说的意思。    
2.使用语法分析提取句子结构。    
3.检测谓语是否有表示说的意思。

#### 选择2. ： 电影评论分类。

在这个作业中你要完成一个电影评论分类任务。

1.数据获取。（采用爬虫技术爬取相关网页上的电影评论数据，例如猫眼电影评论，豆瓣电影评论）

2.把所获得数据分解为训练集，验证集和测试集。

3.选用相应算法构建模型，并测试。

#### 选择3：文章自动续写

在这个作业中你要完成一个文章自动续写的模型。

1.数据获取。（根据你的兴趣采用爬虫技术爬去相关网站上的文本数据内容：比如故事网站，小说网站等）

2.选取模型，并训练。

3.展示一些你模型的输出例子。